<span style="color: #8b1538; font-size: 35px;">**Génomique**</span>

> But

Comment aligner des read sur un génome de référence de manière efficace ?

# Introduction

- Génome humain: 3 400 Mpb - de l'ordre du milliard
- Bactérie type escherichia coli: 4.64 Mpb - de l'ordre du million
- Virus type grippe: 0.013 Mpb

On ne peut donc pas aligner les read sur l'ensemble d'une génome.

<br>

# Solution de base - index sur les kmer

> Principe

- Création du dictionnaire de kmer, i.e l'**index**
- Rechechercher dans l'index

A chaque kmer est associé un ensemble de position dans le génome de réference. Il est donc plus aisé de faire l'alignement avec les read ensuite.

> Problème

- Kmer de taille 1: 4
- Kmer de taille 2: 4^2
- Kmer de taille n: 4^n 

Très rapidement le nombre de kmer possible explose.

<br>

# Transformés de Burrows-wheeler (94)

A l'origine, index pour la compression.

## Exemple

Référence: A T A T C G T

1. Générer l'ensemble des permutations de la séquence de référence - O(n^2)

Le \$ a été choisi car c'est le caréctère juste avant le A - ordre alphabétique.

\begin{matrix}
\$ & A & T & A & T & C & G & T \\
 T &\$ & A & T & A & T & C & G \\
 G & T &\$ & A & T & A & T & C \\
 C & G & T &\$ & A & T & A & T \\
 T & C & G & T &\$ & A & T & A \\
 A & T & C & G & T &\$ & A & T \\
 T & A & T & C & G & T &\$ & A \\
 A & T & A & T & C & G & T &\$
\end{matrix}
 
 2. Trier par ordre alphabétique l'ensemble des permutations obtenues - au mieux O(n(log n)) sinon O(n^2)
 
\begin{matrix}
\$ & A & T & A & T & C & G & T \\
 A & T & A & T & C & G & T &\$ \\
 A & T & C & G & T &\$ & A & T \\
 C & G & T &\$ & A & T & A & T \\
 G & T &\$ & A & T & A & T & C \\
 T &\$ & A & T & A & T & C & G \\
 T & A & T & C & G & T &\$ & A \\
 T & C & G & T &\$ & A & T & A
\end{matrix}
 
 3. Sélectionner la dernière colonne - appelé la transformé de Burrows-wheeler
 
 <span style="color: #8b1538; font-size: 18px;">T \$ T T C G G A A</span>
  
 ## Propriété
 
 - Cette transformé a tendance a regroupé des patternes/mots équivalents ensemble (appelé **motifs**) - compression
 - A partir de la dernière colonne on peut générer la première en la triant dans l'ordre alphabétique (la réciproque est fausse) - toujours stocker la dernière
 - Les éléments d'un même motif sont toujours dans le même ordre quelque soit la colonne que l'on regarde

# TP

Correction: https://drive.google.com/drive/folders/1cMGPhwwUtwBXJgBTZW3lcvrICK_MYZFR

## Algorithme de burrows-wheeler

In [1]:
def burrows_wheeter_transform(seq):
    """
    Cette méthode permet d'appliquer l'algorithme de burrows-wheeter, i.e:
        
        - générer l'ensemble des permutations de la séquence
        - trier par ordre alphabétique l'ensemble des permutations obtenues
        - extraire la dernière colonne
        
    Parameter
    ---------
    seq: str
        la séquence de référence
    
    Return
    ------
    bwt: str
        la séquence extraite, i.e la transformé de burrows-wheeler
    """
    permutations = ['$' + seq]  # la matrice de burrows-wheeler
    for i in range(len(seq)):
        last_element = permutations[i][-1]  # Récupérer le dernier nucléotide de la séquence d'indice i
        tmp = last_element + permutations[i][0:-1]  # Concaténer ce dernier élément à la séquence d'indice i (sans prendre en compte le dernier élément) pour générer la séquence d'indice i+1
        permutations.append(tmp)
    permutations.sort()  # trie par ordre alphabétique
    
    # extraire la dernière colonne
    bwt = ""
    for permut in permutations:
        bwt += permut[-1]  # => bwt = bwt + permut[-1]
    
    return bwt

ref = "ATATCGT"
bwt = burrows_wheeter_transform(ref)
print("La séquence extraite {} à partir de la séquence de référence {} avec l'algorithme de burrows-wheeter".format(bwt, ref))

La séquence extraite T$TTCGAA à partir de la séquence de référence ATATCGT avec l'algorithme de burrows-wheeter


## Inverse burrows-wheeler transform

Principe de la transformé inverse - [Burrows-wheler transform ](https://en.wikipedia.org/wiki/Burrows%E2%80%93Wheeler_transform)

1. Ajout de la séquence transformé en tant que 1ère colonne de la matrice
2. Trie par ordre alphabétique les lignes de la matrice

> Exemple jouet avec la transformé **T\$A**

\begin{matrix}
ADD1 & SORT1 \\
 T   & \$   \\
\$   &  A   \\
 A   &  T  
\end{matrix}

\begin{matrix}
ADD2  & SORT2 \\
 T\$  & \$A   \\
\$A   &  AT   \\
 AT   &  T\$  
\end{matrix}

\begin{matrix}
ADD3  & SORT3 \\
 T\$A & \$AT  \\
\$AT  & AT\$  \\
 AT\$ & T\$A
\end{matrix}

La séquence d'origine est donc **AT**.

In [2]:
def inverse_bwt(bwt):
    """
    Cette méthode permet de revenir à la séquence d'origine à partir de la séquence transformée.
    
        - Ajouter dans inverse la transformé en tant que 1ère colonne
        - Trier par ordre alphabétique les lignes de inverse
    
    Parameter
    ---------
    bwt: str
        la séquence transformée
        
    Return
    ------
    seq: str
        la séquence d'origine
    """
    inverse = [""] * len(bwt)
    for _ in range(len(bwt)):
        tmp = [bwt[i] + inverse[i] for i in range(len(bwt))]  # ajoute la transformé en tant que 1ère colonne
        inverse = sorted(tmp)  # Trie par ordre alphabétique les "ligne" la matrice
    seq = [row for row in inverse if row.startswith('$')][0]  # Extrait la séquence initiale
    return seq.strip('$')

seq_initial = inverse_bwt(bwt)
print("La séquence initiale est {} - trouvé à partir de la transformée {}".format(seq_initial, bwt))

La séquence initiale est ATATCGT - trouvé à partir de la transformée T$TTCGAA


## Génération du FM index

Le [FM index](http://rmpiro.net/teaching/pub/lectures/fu-genomics/05-Read_mapping_2.pdf) (p36) utilise la transformé de burrows-wheeler pour permettre la recherche de patternes dans un string.
Dans notre cas, ca permet d'aligner des reads avec un génome de référence.

Nécessite deux structures de données:
- bwt: la transformée de burrows-wheeler - T\$TTCGAA
- bwt_sort: la transformé de burrows-wheeler dans l'ordre alphabétique \$AACGTTT
        
### Générer la *tally table* de taille m * c avec 

- m: le nombre de position dans la transformée - ici 8
- c: le nombre de charactères - ici 5: \$, A, C, G et T

Une manière efficace de déterminer les nucléotides qui précèdent un nucléotide donné

\begin{matrix}
bwt\_sort & bwt  & \$ & A & C & G & T \\
\$_0      &  T_0 &  0 & 0 & 0 & 0 & 1 \\
 A_0      & \$_0 &  1 & 0 & 0 & 0 & 1 \\
 A_1      &  T_1 &  1 & 0 & 0 & 0 & 2 \\
 C_0      &  T_2 &  1 & 0 & 0 & 0 & 3 \\
 G_0      &  C_0 &  1 & 0 & 1 & 0 & 3 \\
 T_0      &  G_0 &  1 & 0 & 1 & 1 & 3 \\
 T_1      &  A_0 &  1 & 1 & 1 & 1 & 3 \\
 T_2      &  A_1 &  1 & 2 & 1 & 1 & 3
\end{matrix}

In [3]:
import copy

def tally_table(bwt):
    """
    Génère une tally table à partir de la transformée
    
    Parameter
    ---------
    bwt: str
        la séquence transformée
    
    Return
    ------
    tally: list
        la tally table stocké sous la forme d'une liste de dictionnaire
    """
    dico, tally = {'$': 0, 'A': 0, 'C': 0, 'G': 0, 'T': 0}, []
    for nucleotide in bwt:
        dico[nucleotide] += 1  # Une occurence d'un nucleotide donné
        tally.append(copy.deepcopy(dico))  # Ajoute dans la tally table
    return tally
    
tally = tally_table(bwt)
print(tally)

[{'$': 0, 'A': 0, 'C': 0, 'G': 0, 'T': 1}, {'$': 1, 'A': 0, 'C': 0, 'G': 0, 'T': 1}, {'$': 1, 'A': 0, 'C': 0, 'G': 0, 'T': 2}, {'$': 1, 'A': 0, 'C': 0, 'G': 0, 'T': 3}, {'$': 1, 'A': 0, 'C': 1, 'G': 0, 'T': 3}, {'$': 1, 'A': 0, 'C': 1, 'G': 1, 'T': 3}, {'$': 1, 'A': 1, 'C': 1, 'G': 1, 'T': 3}, {'$': 1, 'A': 2, 'C': 1, 'G': 1, 'T': 3}]


### Générer C[c] & ranks avec

- c: les caractères de notre alphabet - \$, A, C, G & T
- C[c]: le nombre d'occurences de charactères lexicalement plus petits dans la séquence et ceux pour chaque caractères
    
    > Soit \$: 0, A: 1, C: 3 -, G: 4, T: 5
    
- ranks: le rang de chaque nucléotide de la transformé

    > Soit T0 $0 T1 T2 C0 G0 A0 A1 - [0, 0, 1, 2, 0, 0, 0, 1]

In [4]:
from collections import Counter

def nb_inferieurs(bwt):
    """
    Génère le nombre d'occurences de caractères plus petits dans la séquence.
    
    Et ceux pour chaque caractères.
    
    Parameter
    ---------
    bwt: str
        la séquence transformée
    
    Return
    ------
    inferieurs: dict
        un dictionnaire des occurences
    """
    inferieurs, count = {'$': 0, 'A': 0, 'C': 0, 'G': 0, 'T': 0}, Counter(sorted(bwt))
    nb = 0
    for nucleotide in inferieurs.keys():
        inferieurs[nucleotide] = nb
        nb += count[nucleotide]
    return inferieurs
    
inferieurs = nb_inferieurs(bwt)


def generer_ranks(bwt):
    """
    Détermine le rang de chaque caractères de la transformée
    
    Parameter
    ---------
    bwt: str
        la séquence transformée
    
    Return
    ------
    ranks: list
        liste des rangs
    """
    dico, ranks = {'$': 0, 'A': 0, 'C': 0, 'G': 0, 'T': 0}, []
    for nucleotide in bwt:
        ranks.append(dico[nucleotide])
        dico[nucleotide] += 1
    return ranks

ranks = generer_ranks(bwt)

print("Nb caractères plus petits: {}".format(inferieurs))
print("Rang: {}".format(ranks))

Nb caractères plus petits: {'$': 0, 'A': 1, 'C': 3, 'G': 4, 'T': 5}
Rang: [0, 0, 1, 2, 0, 0, 0, 1]


### Déterminer les positions des nucléotides de la séquence transformée dans la séquence d'origine

WBT: T0 \$  T1 T2 C0 G0 A0 A1

Seq: A T A T C G T <br>
Pos: 0 1 2 3 4 5 6

T0 -> c'est le dernier T de la séquence car avant \$ - T en pos 6 <br>
T1 -> le 1er T de la séquence - T pos 1 <br>
T2 -> le 2ème T de la séquence - T pos 3 <br>
C0 -> C en position 4 <br>
G0 -> G en position 5 <br>
A1 -> le 1er A de la séquence - A pos 0 <br>
A2 -> le 2ème A de la séquence - A pos 2 

In [5]:
def generer_position(bwt, inferieurs, ranks):
    """
    Détermine la position des nucléotides de la transformée dans la séquence d'origine.
    
    Une position de -1 est affectée au $.
    
    Parameter
    ---------
    bwt: str
        la séquence transformée
    inferieurs: dict
        un dictionnaire des occurences
    ranks: list
        liste des rangs
    
    Return
    ------
    pos: list
        liste des positions
    """
    pos = [-1] * len(bwt)
    seq = ""
    x = len(bwt)-2
    p = 0
    while bwt[p] != "$":
        pos[p] = x
        p = inferieurs[bwt[p]] + ranks[p]
        x -= 1
    return pos

positions = generer_position(bwt, inferieurs, ranks)

print(positions)

[6, -1, 1, 3, 4, 5, 0, 2]


### Alignement read sur génome de référence

La séquence transformée donne comme information

- Les nucléotides présents et leurs nombres
- le nombre et le type d'éléments qui précédent un nucléotide donné

Principe - TAT

1. Sélectionner le dernier nucléotide du read: T
2. Chercher avec la transformée tous les T précédé d'un A
3. Chercher


In [6]:
def map(read, inferieurs, tally):
    """
    Permet d'aligner le read avec le génome de référence.
    
    Parameter
    ---------
    read: str
        le read à aligner
    inferieurs: dict
        un dictionnaire des occurences
    tally: list
        la tally table stocké sous la forme d'une liste de dictionnaire
        
    Return
    ------
    borne_inf: int
        borne inféreur du vecteur positions
    borne_supp: int
        borne supérieur du vecteur positions
    """
    i = len(read) - 1
    borne_inf = inferieurs[read[i]]
    borne_supp = inferieurs[read[i]] + tally[-1][read[i]] - 1
    while i >= 1 and borne_inf <= borne_supp:
        i -= 1
        borne_inf = inferieurs[read[i]] + tally[borne_inf-1][read[i]]
        borne_supp = inferieurs[read[i]] + tally[borne_supp][read[i]] - 1
    return borne_inf, borne_supp

### Méthode d'affichage de l'alignement des reads sur la séquence

Seulement pour des exemple jouets de quelques pb

In [7]:
def determiner_alignement(ref, read, positions, bornes):
    """
    Renvoie l'alignement d'un read donné avec la séquence de référence.

    Parameter
    ---------
    ref: str
        la séquence de référence
    read: str
        le read à aligner
    positions: list
        liste des positions
    bornes: tuple
        les bornes du vecteur positions

    Return
    ------
    alignement: str
        l'alignement du reads avec la séquence de référence
    matches: int
        le nombre de match pour le read donné
    """
    # Déterminer les positions de départ de chaque motifs dans la séquence
    start_motif = []
    for i in range(bornes[0], bornes[1]+1):
        start_motif.append(positions[i]+1)
    start_motif.sort()
    matches = len(start_motif)
    print(start_motif)
    
    # Déterminer l'alignement
    alignement = "." * len(ref)
    for pos in start_motif:
        alignement = alignement[:pos] + read + alignement[pos+len(read):]
        
    return alignement, matches

### Alignement de T sur ATATCGT

In [8]:
read = "T"
bornes = map(read, inferieurs, tally)
print("[Borne inf = {} ; Borne supp = {}]".format(bornes[0], bornes[1]))

[Borne inf = 5 ; Borne supp = 7]


<pre>
<span style="color: blue;">Positions</span> 6 -1 1 3 4 <span style="color: blue; font-weight: bold;">5 0 2</span>
<span style="color: #8b1538;">Index</span>     0  1 2 3 4 <span style="color: #8b1538; font-weight: bold;">5 6 7</span>
</pre>

Indique quand positions 5**+1**, 0**+1** & 2**+1** de la séquence de référence il y a un T

In [9]:
alignement, matches = determiner_alignement(ref, read, positions, bornes)

print("Alignement - {} matches".format(matches))
print("Sequence: {}".format(ref))
print("   Reads: {}".format(alignement))

[1, 3, 6]
Alignement - 3 matches
Sequence: ATATCGT
   Reads: .T.T..T


### Alignement de TAT sur ATATCGT

In [10]:
read = "TAT"
bornes = map(read, inferieurs, tally)
print("[Borne inf = {} ; Borne supp = {}]".format(bornes[0], bornes[1]))

[Borne inf = 6 ; Borne supp = 6]


<pre>
<span style="color: blue;">Positions</span> 6 -1 1 3 4 5 <span style="color: blue; font-weight: bold;">0</span> 2
<span style="color: #8b1538;">Index</span>     0  1 2 3 4 5 <span style="color: #8b1538; font-weight: bold;">6</span> 7
</pre>

Indique quand positions 0**+1** de la séquence de référence il y a un motif **TAT** qui commence.

In [11]:
alignement, matches = determiner_alignement(ref, read, positions, bornes)

print("Alignement - {} matches".format(matches))
print("Sequence: {}".format(ref))
print("   Reads: {}".format(alignement))

[1]
Alignement - 1 matches
Sequence: ATATCGT
   Reads: .TAT...


# Amélioration bwt

> Problème de l'algortihme *Burros-wheeler transform*

Cas de l'algorithme dit naif, création d'une matrice de taille n * n avec n la taille de la séquence de référence.

Dans le cas du génome humain (3 400 Mpb), la taille de la matrice sera de l'ordre du pebibyte (10e18). Il faut donc améliorer l'algorithme de *Burros-wheeler transform*.

> 1ère amélioration "naive"

Lors de la formation de la matrice

In [12]:
import numpy as np

def bwt_amelioration(seq):
    """
    Cette méthode permet d'appliquer l'algorithme de burrows-wheeter tansform.

    Ici, une première approche pour améliorer l'algorithme est mise en place.

    Parameter
    ---------
    seq: str
        la séquence de référence

    Return
    ------
    bwt: str
        la séquence extraite, i.e la transformé de burrows-wheeler
    """
    # Générer la matrice de permutations simplifiée
    permutations, last_element = ["$"], ""
    for i in range(len(seq)):
        last_element += seq[-(i+1)]
        permutations.append(seq[-(i+1):] + "$")
    last_element += "$"

    # Générer la transformée
    sort_permutation = np.argsort(permutations)
    bwt = [""] * len(permutations)
    for i in range(len(bwt)):
        bwt[i] = last_element[sort_permutation[i]]

    return "".join(bwt)

ref = "ATATCGT"
bwt = bwt_amelioration(ref)
print("La séquence extraite {} à partir de la séquence de référence {} avec l'algorithme de burrows-wheeter".format(bwt, ref))

La séquence extraite T$TTCGAA à partir de la séquence de référence ATATCGT avec l'algorithme de burrows-wheeter


Cependant cette méthode bien qu'elle permet de diviser par deux la taille de la matrice de permutaions, on obtient pas un résultat significativement meilleur.